In [39]:
import psycopg2
import requests
import json
import time
from psycopg2.extras import Json
import pandas as pd
import numpy as np

In [40]:
conn = psycopg2.connect("dbname=postgres user=postgres host=localhost port=5432 password=123456")

In [41]:
base_url = 'https://api.hh.ru/vacancies'
params = {
    "text": "NAME:Аналитик",
    # "search_field":[ "exprerience", "schedule"], 
    "exprerience":"noExperience",
    "schedule": 'remote',
    "period": 3,
    "per_page": 100
}

In [42]:
r = requests.get(base_url, params=params)

In [43]:
j = json.loads(r.content)

In [44]:
j['found']

1056

In [45]:
j['items'][4]

{'id': '106670791',
 'premium': False,
 'name': 'Младший аналитик / ресечер (удаленно)',
 'department': None,
 'has_test': False,
 'response_letter_required': False,
 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
 'salary': None,
 'type': {'id': 'open', 'name': 'Открытая'},
 'address': {'city': 'Москва',
  'street': 'Бакунинская улица',
  'building': '73с2',
  'lat': 55.780228,
  'lng': 37.692097,
  'description': None,
  'raw': 'Москва, Бакунинская улица, 73с2',
  'metro': {'station_name': 'Электрозаводская',
   'line_name': 'Арбатско-Покровская',
   'station_id': '3.161',
   'line_id': '3',
   'lat': 55.782057,
   'lng': 37.7053},
  'metro_stations': [{'station_name': 'Электрозаводская',
    'line_name': 'Арбатско-Покровская',
    'station_id': '3.161',
    'line_id': '3',
    'lat': 55.782057,
    'lng': 37.7053}],
  'id': '13056378'},
 'response_url': None,
 'sort_point_distance': None,
 'published_at': '2024-09-02T18:15:45+0300',
 'created_at': '2024-0

In [46]:
vac = j['items'][1]

In [47]:
vac['schedule']

{'id': 'remote', 'name': 'Удаленная работа'}

In [48]:
conn.commit()

In [49]:
def maybe_(schedule):
    try:
        if schedule:
            return schedule.get('id')
        else:
            return None
    except:
        return ''

In [50]:
def get_area(area):
    if area:
        return area['name']
    return None

In [51]:
def salary_from(salary):
    try:
        if salary:
            return salary['from']
        return None
    except:
        return None

In [52]:
def salary_to(salary):
    try:
        if salary:
            return salary['to']
        return None
    except:
        return None

In [53]:
def insert_vacancy(vac):
    cur = conn.cursor()
    cur.execute("insert into analyst values (%s, %s, %s, %s, %s, %s)",
                (vac['id'], vac['name'],
                 get_area(vac['area']),
                 salary_from(vac['salary']),
                 salary_to(vac['salary']),
                 maybe_(vac['schedule'])))
    conn.commit()
    cur.close()

In [54]:
def save_vacancy(page=0):
    print('Downloading page', page)
    r = requests.get(base_url, params=dict(params, page=page))
    time.sleep(1)
    result = json.loads(r.content)
    for v in result['items']:
        insert_vacancy(v)
    if len(result['items']) == 0:
        return
    else:
        save_vacancy(page + 1)

In [55]:
save_vacancy()

In [56]:
cur = conn.cursor()
cur.execute("select id, name, salary_from, salary_to, schedule from analyst limit 20;")
vacs = cur.fetchall()

In [57]:
vacs

[(106662573, 'Аналитик-стажер / Junior Analyst', 65000, None, 'remote'),
 (106707476, 'Координатор-аналитик базы данных', None, 75000, 'remote'),
 (106692739, 'Бизнес-аналитик (стажер)', 60000, 70000, 'remote'),
 (106469611, 'Младший аналитик', None, 75000, 'remote'),
 (106670791, 'Младший аналитик / ресечер (удаленно)', None, None, 'remote'),
 (106694474, 'Системный аналитик Junior', 70000, None, 'remote'),
 (106650793, 'Аналитик баз данных', 75000, None, 'remote'),
 (104461044, 'Младший аналитик данных', 10000, 45000, 'remote'),
 (106501312, 'Ассистент в отдел аналитики', 90000, 120000, 'remote'),
 (106558411, 'Аналитик данных (Python, Pandas, SQL)', 60000, None, 'remote'),
 (106655951, 'Data Analyst', None, None, 'remote'),
 (106722270, 'Продуктовый аналитик в команду каталога', None, None, 'remote'),
 (106632637, 'Junior Data Analyst', None, None, 'remote'),
 (106100439, 'Менеджер – аналитик маркетплейса OZON', 100000, None, 'remote'),
 (106691744, 'Аналитик данных', None, None, 'r

In [58]:
#######################

In [61]:
an2 = pd.DataFrame(vacs)

In [62]:
an2.to_csv('analysts_hh_0310.csv', index=False)

In [64]:
an2.head(30)

,0,1,2,3,4
0,106662573,Аналитик-стажер / Junior Analyst,65000.0,NaN,remote
1,106707476,Координатор-аналитик базы данных,NaN,75000.0,remote
2,106692739,Бизнес-аналитик (стажер),60000.0,70000.0,remote
3,106469611,Младший аналитик,NaN,75000.0,remote
4,106670791,Младший аналитик / ресечер (удаленно),NaN,NaN,remote
5,106694474,Системный аналитик Junior,70000.0,NaN,remote
6,106650793,Аналитик баз данных,75000.0,NaN,remote
7,104461044,Младший аналитик данных,10000.0,45000.0,remote
8,106501312,Ассистент в отдел аналитики,90000.0,120000.0,remote
9,106558411,"Аналитик данных (Python, Pandas, SQL)",60000.0,NaN,remote
